<a href="https://colab.research.google.com/github/u-masao/vector-search-tutorial-for-japanese/blob/main/Session04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 日本語のためのベクトル検索ハンズオン：Session 4

このセッションでは、Qdrant の実践的な使い方を確認します。

Docker で Qdrant を実行して、Python SDK で基本的な操作をします。

## 環境構築


### Google Colab 向け Docker インストール

In [1]:
# Copyright 2024 Drengskapur
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#
# @title {display-mode:"form"}
# @markdown <br/><br/><center><img src="https://cdn.jsdelivr.net/gh/drengskapur/docker-in-colab/assets/docker.svg" height="150"><img src="https://cdn.jsdelivr.net/gh/drengskapur/docker-in-colab/assets/colab.svg" height="150"></center><br/>
# @markdown <center><h1>Docker in Colab</h1></center><center>github.com/drengskapur/docker-in-colab<br/><br/><br/><b>udocker("run hello-world")</b></center><br/>
def udocker_init():
    import os
    if not os.path.exists("/home/user"):
        !pip install udocker > /dev/null
        !udocker --allow-root install > /dev/null
        !useradd -m user > /dev/null
    print(f'Docker-in-Colab 1.1.0\n')
    print(f'Usage:     udocker("--help")')
    print(f'Examples:  https://github.com/indigo-dc/udocker?tab=readme-ov-file#examples')

    def execute(command: str):
        user_prompt = "\033[1;32muser@pc\033[0m"
        print(f"{user_prompt}$ udocker {command}")
        !su - user -c "udocker $command"

    return execute

udocker = udocker_init()

Docker-in-Colab 1.1.0

Usage:     udocker("--help")
Examples:  https://github.com/indigo-dc/udocker?tab=readme-ov-file#examples


In [2]:
# udocker("pull qdrant/qdrant")

In [3]:
udocker("run -p 127.0.0.1:6333:6333 -p 127.0.0.1:6334:6334 qdrant/qdrant")

# 初回実行時は正常に終了しないことがあります。しばらくして応答がなければランタイムをリセットしてください。

user@pc$ udocker run -p 127.0.0.1:6333:6333 -p 127.0.0.1:6334:6334 qdrant/qdrant
 
 ****************************************************************************** 
 *                                                                            * 
 *               STARTING 946eaae4-8c60-3d3e-8f96-de5d6e7c54f4                * 
 *                                                                            * 
 ****************************************************************************** 
 executing: entrypoint.sh
           _                 _    
  __ _  __| |_ __ __ _ _ __ | |_  
 / _` |/ _` | '__/ _` | '_ \| __| 
| (_| | (_| | | | (_| | | | | |_  
 \__, |\__,_|_|  \__,_|_| |_|\__| 
    |_|                           

Version: 1.8.4, build: 984f55d6
Access web UI at http://localhost:6333/dashboard

2024-04-21T14:37:18.951287Z  INFO storage::content_manager::consensus::persistent: Initializing new raft state at ./storage/raft_state.json    
2024-04-21T14:37:19.557545Z  INFO qdrant: Distrib

###  SageMaker 向け docker 実行

!docker run -p 127.0.0.1:6333:6333 qdrant/qdranbt
(省略)

## パッケージのインストールと読み込み

In [4]:
!pip install -q qdrant-client sentence-transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.2/223.2 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 42.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [5]:
from sentence_transformers import SentenceTransformer
from datasets import load_dataset
import pandas as pd
import numpy as np

## Qdrant の利用

以下を実行します。

- ベクトル作成
  - データ取得
  - テキスト作成
  - 埋め込み計算
- Qdrant Client を初期化
- コレクションを初期化(Cosine計算モード)
- ポイント(アイテム)を登録
  - ベクトル、ペイロード、インデックス
- ベクトル検索のクエリ例
- ベクトル検索&フィルタリングのクエリ例

### データの読み込みと埋め込みベクトルの計算

In [6]:
# Hugging Face からデータセットを取得
# データセットの説明 → https://huggingface.co/datasets/sbintuitions/JMTEB
dataset = load_dataset("sbintuitions/JMTEB", name="livedoor_news")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for sbintuitions/JMTEB contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/sbintuitions/JMTEB
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  wa

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['url', 'timestamp', 'title', 'text', 'label'],
        num_rows: 5163
    })
    validation: Dataset({
        features: ['url', 'timestamp', 'title', 'text', 'label'],
        num_rows: 1106
    })
    test: Dataset({
        features: ['url', 'timestamp', 'title', 'text', 'label'],
        num_rows: 1107
    })
})

In [8]:
dataset['train'].features

{'url': Value(dtype='string', id=None),
 'timestamp': Value(dtype='string', id=None),
 'title': Value(dtype='string', id=None),
 'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=('dokujo-tsushin', 'it-life-hack', 'kaden-channel', 'livedoor-homme', 'movie-enter', 'peachy', 'smax', 'sports-watch', 'topic-news'), id=None)}

In [9]:
label_names = dataset['train'].features['label'].names
text_df  = pd.DataFrame(dataset['train']).sample(100, random_state = 12345)  # 100件をランダムに抽出
text_df['label_name'] = text_df['label'].apply(lambda x: label_names[x])
text_df.sort_values('url')

,url,timestamp,title,text,label,label_name
4383,http://news.livedoor.com/article/detail/4511697/,2009-12-18T19:00:00+0900,10回目のテーマは“GIRLS’ LAND”！ 東京ガールズコレクション、舞台を横浜アリーナへ,\n今回、TGCではこれまでの感謝と、「トレンドに敏感な女の子たちが、どんなときも常に前向き...,5,peachy
4410,http://news.livedoor.com/article/detail/4515083/,2009-12-21T15:30:00+0900,ガチャピン・ムック、世界一のパティシエ辻口氏と夢のコラボ！,\n本イベントは、詩人の谷川俊太郎氏によるガチャピン・ムックの誕生秘話を描いた絵本「ガチャピ...,5,peachy
3809,http://news.livedoor.com/article/detail/4546120/,2010-01-14T11:10:00+0900,インタビュー：インナービューティスペシャリスト・友野なお「自分で体験したことを伝えたい」,\n——野菜ソムリエの資格を取得されインナービューティスペシャリストとして様々なメディアでご...,5,peachy
277,http://news.livedoor.com/article/detail/4663550/,2010-03-18T15:45:00+0900,インタビュー：エッセイスト・上原愛加さん「甘い物と幸せがなくては、生きて行けません（笑）」,\n--エッセイストになられたきっかけはありますか？\n上原愛加（以下、上原）：高校生の頃か...,5,peachy
4625,http://news.livedoor.com/article/detail/4720642/,2010-04-21T10:00:00+0900,3人に1人が参加経験有り？！ 今ドキ出会いの現場に潜入,\n出会いと別れの季節だからこそ、新たな出会いに心躍らせてしまう女子は多いはず。出会いの場と...,5,peachy
...,...,...,...,...,...,...
3571,http://news.livedoor.com/article/detail/6853693/,2012-08-14T12:50:00+0900,【終了しました】クレンジングリサーチの姉妹ブランド「スキンケアリサーチ」のジェリー美容液を5...,\n「プランプスキンモイスチャージェリー美容液」は、蜜りんご保湿成分や角層浸透型Wコラーゲン...,5,peachy
1881,http://news.livedoor.com/article/detail/6856211/,2012-08-15T10:00:00+0900,Googleロゴが往年の料理研究家に！ 本日のGoogleロゴは有名な料理研究家の生誕100...,\nズンドウみたいなモノから始まり、Googleという文字になるようにナベや食材が並んでいる...,1,it-life-hack
4620,http://news.livedoor.com/article/detail/6856582/,2012-08-15T11:30:00+0900,握手会やイベント参加…「アイドル戦国時代」に独女も大量参戦,「そもそもアイドル戦国時代」って何なの？ という人に補足しておこう。2010年ごろから始まっ...,0,dokujo-tsushin
3353,http://news.livedoor.com/article/detail/6886811/,2012-08-24T19:44:00+0900,パナソニック、ミラーレスタイプのデジタル一眼カメラ「ＤＭＣ−Ｇ５」を発売【売れ筋チェック】,デジタル一眼カメラ市場は、コンパクトカメラからのステップアップで表現力豊かな一眼写真を楽しむ...,2,kaden-channel


In [10]:
def build_concat_text(row):
    return f'''
        # タイトル

        {row['title']}

        ## ニュースカテゴリ

        {row['label_name']}

        ## 記事

        {row['text']}
    '''

# 埋め込み対象テキストを作成
text_df['embed_text'] = text_df.apply(build_concat_text, axis=1)

# ライセンス表記の行を削除
text_df = text_df[~text_df['url'].str.startswith('このディレクトリにあるすべての記事ファイル')]

# タイトルの長さを計算
text_df['title_length'] = text_df['title'].str.len()

# 記事本体の長さを計算
text_df['text_length'] = text_df['text'].str.len()

# URL からアイテムIDを計算
text_df['id'] = text_df['url'].replace('.*detail/([0-9]*)/',r'\1',regex=True).astype(int)

text_df.sort_values('id')

,url,timestamp,title,text,label,label_name,embed_text,title_length,text_length,id
4383,http://news.livedoor.com/article/detail/4511697/,2009-12-18T19:00:00+0900,10回目のテーマは“GIRLS’ LAND”！ 東京ガールズコレクション、舞台を横浜アリーナへ,\n今回、TGCではこれまでの感謝と、「トレンドに敏感な女の子たちが、どんなときも常に前向き...,5,peachy,\n # タイトル\n\n 10回目のテーマは“GIRLS’ L...,47,1158,4511697
4410,http://news.livedoor.com/article/detail/4515083/,2009-12-21T15:30:00+0900,ガチャピン・ムック、世界一のパティシエ辻口氏と夢のコラボ！,\n本イベントは、詩人の谷川俊太郎氏によるガチャピン・ムックの誕生秘話を描いた絵本「ガチャピ...,5,peachy,\n # タイトル\n\n ガチャピン・ムック、世界一のパティシ...,29,791,4515083
3809,http://news.livedoor.com/article/detail/4546120/,2010-01-14T11:10:00+0900,インタビュー：インナービューティスペシャリスト・友野なお「自分で体験したことを伝えたい」,\n——野菜ソムリエの資格を取得されインナービューティスペシャリストとして様々なメディアでご...,5,peachy,\n # タイトル\n\n インタビュー：インナービューティスペ...,44,1723,4546120
277,http://news.livedoor.com/article/detail/4663550/,2010-03-18T15:45:00+0900,インタビュー：エッセイスト・上原愛加さん「甘い物と幸せがなくては、生きて行けません（笑）」,\n--エッセイストになられたきっかけはありますか？\n上原愛加（以下、上原）：高校生の頃か...,5,peachy,\n # タイトル\n\n インタビュー：エッセイスト・上原愛加...,45,1250,4663550
4625,http://news.livedoor.com/article/detail/4720642/,2010-04-21T10:00:00+0900,3人に1人が参加経験有り？！ 今ドキ出会いの現場に潜入,\n出会いと別れの季節だからこそ、新たな出会いに心躍らせてしまう女子は多いはず。出会いの場と...,5,peachy,\n # タイトル\n\n 3人に1人が参加経験有り？！ 今ドキ...,27,1418,4720642
...,...,...,...,...,...,...,...,...,...,...
3571,http://news.livedoor.com/article/detail/6853693/,2012-08-14T12:50:00+0900,【終了しました】クレンジングリサーチの姉妹ブランド「スキンケアリサーチ」のジェリー美容液を5...,\n「プランプスキンモイスチャージェリー美容液」は、蜜りんご保湿成分や角層浸透型Wコラーゲン...,5,peachy,\n # タイトル\n\n 【終了しました】クレンジングリサーチ...,54,1306,6853693
1881,http://news.livedoor.com/article/detail/6856211/,2012-08-15T10:00:00+0900,Googleロゴが往年の料理研究家に！ 本日のGoogleロゴは有名な料理研究家の生誕100...,\nズンドウみたいなモノから始まり、Googleという文字になるようにナベや食材が並んでいる...,1,it-life-hack,\n # タイトル\n\n Googleロゴが往年の料理研究家に...,50,637,6856211
4620,http://news.livedoor.com/article/detail/6856582/,2012-08-15T11:30:00+0900,握手会やイベント参加…「アイドル戦国時代」に独女も大量参戦,「そもそもアイドル戦国時代」って何なの？ という人に補足しておこう。2010年ごろから始まっ...,0,dokujo-tsushin,\n # タイトル\n\n 握手会やイベント参加…「アイドル戦国...,29,1946,6856582
3353,http://news.livedoor.com/article/detail/6886811/,2012-08-24T19:44:00+0900,パナソニック、ミラーレスタイプのデジタル一眼カメラ「ＤＭＣ−Ｇ５」を発売【売れ筋チェック】,デジタル一眼カメラ市場は、コンパクトカメラからのステップアップで表現力豊かな一眼写真を楽しむ...,2,kaden-channel,\n # タイトル\n\n パナソニック、ミラーレスタイプのデジ...,45,535,6886811


In [11]:
text_df.describe()

,label,title_length,text_length,id
count,100.000000,100.000000,100.000000,1.000000e+02
mean,4.140000,37.810000,987.380000,6.017434e+06
std,2.428867,13.417684,597.492146,6.631376e+05
min,0.000000,16.000000,146.000000,4.511697e+06
25%,2.000000,28.000000,572.000000,5.692977e+06
50%,4.000000,35.500000,868.000000,6.152064e+06
75%,6.000000,45.000000,1303.750000,6.548404e+06
max,8.000000,82.000000,4225.000000,6.896226e+06


In [12]:
# 埋め込みモデルを初期化
#   日本語の埋め込みで評判が良いモデルを利用
#   https://huggingface.co/intfloat/multilingual-e5-small
model = SentenceTransformer("intfloat/multilingual-e5-small")

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [13]:
# 埋め込みモデルの出力次元酢を取得
embedding_dim = model.get_sentence_embedding_dimension()
print("Embedding dimension:", embedding_dim)

Embedding dimension: 384


In [14]:
# タイトルと本文を結合したテキストをベクトル化
# 最初の512トークン分までを反映
vectors = model.encode(text_df['embed_text'].values)

In [15]:
# 埋め込みベクトルをファイルに保存
np.save("vectors.npy", vectors)

In [16]:
# 取得した埋め込みベクトルの概要
print("ndarray.shape:",vectors.shape)
print("各ベクトルのノルム(L2)の算術平均:",np.linalg.norm(vectors, ord=2, axis=0).mean())

ndarray.shape: (100, 384)
各ベクトルのノルム(L2)の算術平均: 0.48115888


In [17]:
# サンプル表示
vectors[0]

array([ 3.87505442e-02, -1.96494572e-02, -8.51922631e-02, -6.66290149e-02,
        3.79392244e-02, -5.51324300e-02,  1.69071425e-02,  2.62583196e-02,
        5.09026907e-02,  2.53173169e-02,  5.13894856e-02,  1.29771773e-02,
        7.89590701e-02,  1.28859552e-02, -5.47156036e-02,  2.98775341e-02,
        6.88948035e-02, -2.96184104e-02, -7.90087134e-02, -5.16211726e-02,
        3.62532213e-02,  2.93280128e-02, -6.35307059e-02,  4.75678667e-02,
        6.33137450e-02,  3.29403579e-02, -2.90594790e-02,  1.27268150e-01,
        1.28972605e-02, -4.98516075e-02, -6.79600388e-02, -8.77954662e-02,
        8.33689868e-02, -5.47719300e-02,  3.88767906e-02, -5.04318718e-03,
       -7.68378451e-02, -2.27394234e-02,  5.51715866e-03, -6.14457689e-02,
       -6.68925419e-02,  6.10201210e-02,  1.73050035e-02,  1.07135899e-01,
        4.93336730e-02,  8.13406259e-02, -2.86618862e-02,  6.69908300e-02,
       -4.29456271e-02, -1.74532328e-02, -1.43488990e-02,  4.96156290e-02,
       -1.90618504e-02,  

## Qdrant Client を初期化

In [18]:
from qdrant_client import QdrantClient

qdrant_url = "http://localhost:6333"
qdrant_collection_name = "livedoor_news"

# Qdrant Client を初期化
client = QdrantClient(url=qdrant_url)

In [19]:
# コレクションの情報を取得
# status = client.get_collection( collection_name=qdrant_collection_name )
# print(status)

# コレクションを削除
# status = client.delete_collection( collection_name=qdrant_collection_name )
# print(status)

In [20]:
from qdrant_client.models import Distance, VectorParams

# コレクション(テーブル)を初期化
# ベクトル次元 4次元、距離: cosine
# (既にコレクションが定義されている場合はエラーが出ます)
#  -> b'{"status":{"error":"Wrong input: Collection `test_collection` already exists!"},"time":0.0000995}'
client.create_collection(
    collection_name=qdrant_collection_name,
    vectors_config=VectorParams(size=embedding_dim, distance=Distance.COSINE),
)

True

In [21]:
# ペイロードのインデックスのスキーマを定義
# None の場合はインデックスを作成しない
payload_index_field_types = {
    "id":"integer",
    "url":None,
    "timestamp":"datetime",
    "label_name":"keyword",
    "label":"integer",
    "text_length":"integer",
}

In [22]:
from qdrant_client.models import PointStruct, Batch

# ポイント(アイテム)を追加
# ベクトルとペイロードを追加
operation_info = client.upsert(
    collection_name=qdrant_collection_name,
    wait=True,  # 検索可能な状態になるまで応答を待つ
    points=Batch(
        ids=text_df['id'].to_list(),
        payloads=text_df[payload_index_field_types.keys()].to_dict('records'),
        vectors=vectors
    )
)

print(operation_info)

operation_id=0 status=<UpdateStatus.COMPLETED: 'completed'>


In [23]:
# set index
for field_name, field_type in payload_index_field_types.items():
    if field_type is None:
        continue
    print(f"field_name: {field_name}, field_type: {field_type}")
    client.create_payload_index(
        collection_name=qdrant_collection_name,
        field_name=field_name,
        field_schema=field_type,
    )

field_name: id, field_type: integer
field_name: timestamp, field_type: datetime
field_name: label_name, field_type: keyword
field_name: label, field_type: integer
field_name: text_length, field_type: integer


## 検索

In [24]:
# クエリ文字列からクエリベクトルを作成
query_text = 'インフラ関連の整備'
query_vector = model.encode(query_text)

In [25]:
query_vector.shape

(384,)

In [26]:
# クエリベクトルに対するベクトル検索(Cosine類似度)
search_result = client.search(
    collection_name=qdrant_collection_name,
    query_vector=query_vector,  # 検索キーベクトルを指定
    with_payload=True, # 応答にペイロードを含める
    limit=5
)

_ = [print(x) for x in search_result]

id=6786794 version=0 score=0.8213565 payload={'id': 6786794, 'label': 6, 'label_name': 'smax', 'text_length': 1651, 'timestamp': '2012-07-24T16:55:00+0900', 'url': 'http://news.livedoor.com/article/detail/6786794/'} vector=None shard_key=None
id=6324538 version=0 score=0.81743443 payload={'id': 6324538, 'label': 1, 'label_name': 'it-life-hack', 'text_length': 363, 'timestamp': '2012-02-29T18:39:00+0900', 'url': 'http://news.livedoor.com/article/detail/6324538/'} vector=None shard_key=None
id=6431890 version=0 score=0.8163908 payload={'id': 6431890, 'label': 2, 'label_name': 'kaden-channel', 'text_length': 540, 'timestamp': '2012-04-03T13:15:00+0900', 'url': 'http://news.livedoor.com/article/detail/6431890/'} vector=None shard_key=None
id=6317547 version=0 score=0.8133148 payload={'id': 6317547, 'label': 2, 'label_name': 'kaden-channel', 'text_length': 1094, 'timestamp': '2012-02-27T15:00:00+0900', 'url': 'http://news.livedoor.com/article/detail/6317547/'} vector=None shard_key=None
id=

In [27]:
from qdrant_client.models import Filter, FieldCondition, MatchValue

# ベクトル検索 & フィルタリングの検索例
search_result = client.search(
    collection_name=qdrant_collection_name,
    query_vector=query_vector,  # 検索キーベクトルを指定
    query_filter=Filter(
        must=[FieldCondition(key="label_name", match=MatchValue(value="it-life-hack"))]  # ペイロードの文字列一致でフィルタリング
    ),
    with_payload=True, # 応答にペイロードを含める
    limit=5,
)

_ = [print(x) for x in search_result]

id=6324538 version=0 score=0.81743443 payload={'id': 6324538, 'label': 1, 'label_name': 'it-life-hack', 'text_length': 363, 'timestamp': '2012-02-29T18:39:00+0900', 'url': 'http://news.livedoor.com/article/detail/6324538/'} vector=None shard_key=None
id=6306744 version=0 score=0.8088398 payload={'id': 6306744, 'label': 1, 'label_name': 'it-life-hack', 'text_length': 1102, 'timestamp': '2012-02-23T16:00:00+0900', 'url': 'http://news.livedoor.com/article/detail/6306744/'} vector=None shard_key=None
id=6540729 version=0 score=0.80669 payload={'id': 6540729, 'label': 1, 'label_name': 'it-life-hack', 'text_length': 1715, 'timestamp': '2012-05-09T10:00:00+0900', 'url': 'http://news.livedoor.com/article/detail/6540729/'} vector=None shard_key=None
id=6723561 version=0 score=0.798377 payload={'id': 6723561, 'label': 1, 'label_name': 'it-life-hack', 'text_length': 1403, 'timestamp': '2012-07-04T13:00:00+0900', 'url': 'http://news.livedoor.com/article/detail/6723561/'} vector=None shard_key=None

In [28]:
from qdrant_client.models import Filter, FieldCondition, Range

# ベクトル検索 & フィルタリングの検索例
search_result = client.search(
    collection_name=qdrant_collection_name,
    query_vector=query_vector,  # 検索キーベクトルを指定
    query_filter=Filter(
        must=[
            FieldCondition(
                key="text_length",
                range=Range(  # ペイロードの値の範囲でフィルタリング
                    gte=600,
                    lte=1000,
                ),
            ),
        ]
    ),
    with_payload=True, # 応答にペイロードを含める
    limit=10,
)

_ = [print(x) for x in search_result]

id=5795649 version=0 score=0.8108609 payload={'id': 5795649, 'label': 2, 'label_name': 'kaden-channel', 'text_length': 846, 'timestamp': '2011-08-19T12:00:00+0900', 'url': 'http://news.livedoor.com/article/detail/5795649/'} vector=None shard_key=None
id=6316064 version=0 score=0.8071237 payload={'id': 6316064, 'label': 2, 'label_name': 'kaden-channel', 'text_length': 856, 'timestamp': '2012-02-27T10:00:00+0900', 'url': 'http://news.livedoor.com/article/detail/6316064/'} vector=None shard_key=None
id=5913813 version=0 score=0.8068694 payload={'id': 5913813, 'label': 3, 'label_name': 'livedoor-homme', 'text_length': 752, 'timestamp': '2011-10-05T14:05:00+0900', 'url': 'http://news.livedoor.com/article/detail/5913813/'} vector=None shard_key=None
id=6666132 version=0 score=0.80412257 payload={'id': 6666132, 'label': 2, 'label_name': 'kaden-channel', 'text_length': 880, 'timestamp': '2012-06-17T13:30:00+0900', 'url': 'http://news.livedoor.com/article/detail/6666132/'} vector=None shard_key

In [31]:

from qdrant_client.models import Filter, FieldCondition, MatchValue

# ベクトル検索 & フィルタリングの検索例
# 2つのフィルタリング条件を And で結合
search_result = client.search(
    collection_name=qdrant_collection_name,
    query_vector=query_vector,  # 検索キーベクトルを指定
    query_filter=Filter(
        must=[
            FieldCondition(  # ペイロードの文字列一致でフィルタリング
                key="label_name",
                match=MatchValue(value="it-life-hack")
            ),
            FieldCondition(  # ペイロードの値の範囲でフィルタリング
                key="text_length",
                range=Range( gte=600, lte=1700,
                ),
            ),
        ]
    ),
    with_payload=True,  # ペイロードを取得
    limit=5,
)

_ = [print(x) for x in search_result]

id=6306744 version=0 score=0.8088398 payload={'id': 6306744, 'label': 1, 'label_name': 'it-life-hack', 'text_length': 1102, 'timestamp': '2012-02-23T16:00:00+0900', 'url': 'http://news.livedoor.com/article/detail/6306744/'} vector=None shard_key=None
id=6723561 version=0 score=0.798377 payload={'id': 6723561, 'label': 1, 'label_name': 'it-life-hack', 'text_length': 1403, 'timestamp': '2012-07-04T13:00:00+0900', 'url': 'http://news.livedoor.com/article/detail/6723561/'} vector=None shard_key=None
id=6547890 version=0 score=0.7905753 payload={'id': 6547890, 'label': 1, 'label_name': 'it-life-hack', 'text_length': 1612, 'timestamp': '2012-05-11T09:00:00+0900', 'url': 'http://news.livedoor.com/article/detail/6547890/'} vector=None shard_key=None
id=6601550 version=0 score=0.7905595 payload={'id': 6601550, 'label': 1, 'label_name': 'it-life-hack', 'text_length': 1597, 'timestamp': '2012-05-28T13:00:00+0900', 'url': 'http://news.livedoor.com/article/detail/6601550/'} vector=None shard_key=No